Righnow candidate is : SVR with PCA n_components = 10 with results 0.23706564506631833

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test.head(10)

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
5,8,y,aa,ai,e,d,x,g,s,0,...,1,0,0,0,0,0,0,0,0,0
6,10,x,b,ae,d,d,x,d,y,0,...,0,0,0,0,0,1,0,0,0,0
7,11,f,s,ae,c,d,h,d,a,0,...,0,0,1,0,0,0,0,0,0,0
8,12,ap,l,s,c,d,h,j,n,0,...,0,0,0,0,0,0,0,0,0,0
9,14,o,v,as,f,d,g,f,v,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel

In [5]:
train_labels = train.iloc[:,1]
train_feats = train.iloc[:, 2:]

test_labels = test.iloc[:,1]
test_feats = test.iloc[:, 1:]

In [6]:
train_feats_encode = []
train_feats_encode.append(train.iloc[:, 0])
for i in range(train_feats.shape[1]):
    arr = train_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    train_feats_encode.append(arr)
train_feats_encode = np.array(train_feats_encode).T

test_feats_encode = []
test_feats_encode.append(test.iloc[:, 0])
for i in range(test_feats.shape[1]):
    arr = test_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    test_feats_encode.append(arr)
test_feats_encode = np.array(test_feats_encode).T

In [7]:
train_X, test_X, train_y, test_y = train_test_split(train_feats_encode, train_labels)


*** Keras Deep Learning ***

In [8]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
act_func = 'tanh'
input_dims = train_feats_encode.shape[1]

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

Using TensorFlow backend.


In [9]:
# base model architecture definition
def model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [10]:
estimator = KerasRegressor(
    build_fn=model, 
    nb_epoch=100, 
    batch_size=20,
    verbose=1
)

In [11]:
estimator.fit(
    train_X, 
    train_y, 
    epochs=300,
    verbose=2,
    shuffle=True
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 377)               142506    
_________________________________________________________________
batch_normalization_1 (Batch (None, 377)               1508      
_________________________________________________________________
activation_1 (Activation)    (None, 377)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 377)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 377)               142506    
_________________________________________________________________
batch_normalization_2 (Batch (None, 377)               1508      
_________________________________________________________________
activation_2 (Activation)    (None, 377)               0         
__________

1s - loss: 76.6265 - r2_keras: 0.5424
Epoch 117/300
1s - loss: 78.5559 - r2_keras: 0.5356
Epoch 118/300
1s - loss: 77.5598 - r2_keras: 0.5451
Epoch 119/300
1s - loss: 80.5612 - r2_keras: 0.5191
Epoch 120/300
1s - loss: 80.6797 - r2_keras: 0.5087
Epoch 121/300
1s - loss: 77.1891 - r2_keras: 0.5503
Epoch 122/300
1s - loss: 77.2892 - r2_keras: 0.5454
Epoch 123/300
1s - loss: 80.1053 - r2_keras: 0.5137
Epoch 124/300
1s - loss: 77.7255 - r2_keras: 0.5451
Epoch 125/300
1s - loss: 77.3926 - r2_keras: 0.5429
Epoch 126/300
1s - loss: 78.7195 - r2_keras: 0.5382
Epoch 127/300
1s - loss: 80.0838 - r2_keras: 0.5189
Epoch 128/300
1s - loss: 79.1447 - r2_keras: 0.5361
Epoch 129/300
1s - loss: 76.7192 - r2_keras: 0.5504
Epoch 130/300
1s - loss: 76.9296 - r2_keras: 0.5535
Epoch 131/300
1s - loss: 79.3671 - r2_keras: 0.5253
Epoch 132/300
1s - loss: 77.6643 - r2_keras: 0.5467
Epoch 133/300
1s - loss: 77.9699 - r2_keras: 0.5361
Epoch 134/300
1s - loss: 76.7313 - r2_keras: 0.5390
Epoch 135/300
1s - loss: 7

1s - loss: 74.2410 - r2_keras: 0.5674
Epoch 275/300
1s - loss: 72.6398 - r2_keras: 0.5767
Epoch 276/300
1s - loss: 72.5429 - r2_keras: 0.5818
Epoch 277/300
1s - loss: 74.4185 - r2_keras: 0.5563
Epoch 278/300
1s - loss: 73.7198 - r2_keras: 0.5740
Epoch 279/300
1s - loss: 73.5051 - r2_keras: 0.5676
Epoch 280/300
1s - loss: 72.7103 - r2_keras: 0.5831
Epoch 281/300
1s - loss: 71.2542 - r2_keras: 0.5921
Epoch 282/300
1s - loss: 72.0490 - r2_keras: 0.5724
Epoch 283/300
1s - loss: 71.8232 - r2_keras: 0.5838
Epoch 284/300
1s - loss: 73.3976 - r2_keras: 0.5666
Epoch 285/300
1s - loss: 72.8837 - r2_keras: 0.5732
Epoch 286/300
1s - loss: 73.3723 - r2_keras: 0.5642
Epoch 287/300
1s - loss: 74.3858 - r2_keras: 0.5603
Epoch 288/300
1s - loss: 73.8659 - r2_keras: 0.5570
Epoch 289/300
1s - loss: 72.6223 - r2_keras: 0.5841
Epoch 290/300
1s - loss: 74.4676 - r2_keras: 0.5690
Epoch 291/300
1s - loss: 72.4378 - r2_keras: 0.5773
Epoch 292/300
1s - loss: 73.5461 - r2_keras: 0.5766
Epoch 293/300
1s - loss: 7

In [ ]:
prediction_dl  = estimator.predict(train_X)
test_dl = estimator.predict(test_X)
score = r2_score(test_y, test_dl)
print("\ntrain : ", r2_score(train_y, prediction_dl)) 
print("\ntest : ", score) 

In [ ]:
estimator.fit(
    train_X, 
    train_y, 
    epochs=300,
    verbose=0,
    shuffle=True
)

In [ ]:
test_predictions = estimator.predict(test_feats_encode)

In [ ]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': test_predictions})
output.to_csv('results/ensemble[%s][%s].csv' % (str(model.__class__.__name__),score), index=False)
